## Compile taxonomy assignments
Following assigning taxonomy to reference sequences, compile with exisiting ASV or OTU table

In [5]:
library(Biostrings)

Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required p

In [14]:
# Import reference sequences

## ASVs
ref_asv <- readDNAStringSet("/vortexfs1/omics/huber/shu/slo-pier-weekly/qiime2/asv/slo-pier-ref-seqs-asv.fna")
Feature.ID <- names(ref_asv)
ReferenceSequence <- paste(ref_asv)
fna_df <- data.frame(Feature.ID, ReferenceSequence)
# head(fna_df)

## OTUs
ref_otu <- readDNAStringSet("/vortexfs1/omics/huber/shu/slo-pier-weekly/qiime2/otu/slo-pier-ref-seqs-otu.fna")
Feature.ID <- names(ref_otu)
ReferenceSequence <- paste(ref_otu)
fna_df_otu <- data.frame(Feature.ID, ReferenceSequence)
# head(fna_df_otu)

In [ ]:
# library(dada2)
# seqs <- as.character(fna_df$ReferenceSequence) #extract sequences
# taxa_pr2 <- assignTaxonomy(seqs, "/vortexfs1/omics/huber/shu/db/pr2-db/pr2_version_4.12.0_18S_dada2.fasta.gz",
#         taxLevels = c("Kingdom","Supergroup","Division","Class","Order","Family","Genus","Species"), multithread = TRUE, minBoot = 0)

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [4]:
# Import pr2 assigned taxonomy output (minBoot = 0) and original count tables
load("Pier-assigned-refseqs.RData", verbose = T)
asv_table <- read.delim("/vortexfs1/omics/huber/shu/slo-pier-weekly/qiime2/asv/CountTable-wtax-2020-04-22.txt")
otu_table <- read.delim("/vortexfs1/omics/huber/shu/slo-pier-weekly/qiime2/otu/CountTable-wtax-2020-07-02.txt")

Loading objects:
  taxa_pr2
  taxa_pr2_otu


In [18]:
head(otu_table)

Feature.ID,AP.AV17.AV180216.18S,AP.AV23.AV180316.18S,AV12.18S,AV13.18S,AV14.18S,AV17.2.18S,AV18.18S,AV19.18S,AV22.18S,...,AV84.18S,AV90.18S,AP.AV29.AV180420.18S,AP.AV41.AV180518.18S,AV32.18S,AV35.18S,AV56.18S,NEGPCR18SControl.18S,Taxon,Confidence
c145e8099f5a2f17debf82f9bbda6db5a220543e,584,284,1172,1619,910,3153,1249,1325,1684,...,902,5656,2,0,2,0,3,0,Eukaryota;Alveolata;Dinoflagellata;Dinophyceae;Gymnodiniales;Gymnodiniaceae;Margalefidinium;Margalefidinium_fulvescens;,0.8200643
2b17741c943bfa5057c2ba8ddced03eacd595b41,2000,1246,1475,13380,6123,17999,1466,3958,35444,...,3063,15725,9449,5928,50249,2318,21,0,Eukaryota;Alveolata;Dinoflagellata;Dinophyceae,0.9999630
a940c0f810f1168ddf166d4f9e17e069a1036719,117,17,1028,98,0,363,1205,1495,130,...,17,6476,2,510,628,214,1,0,Eukaryota;Archaeplastida,0.9964566
59f6fc378a02095eb9a6c5fa614439f511ce9ccb,193,87,3622,628,30,918,2823,5533,880,...,144,15365,6,95,346,168,1,0,Eukaryota;Archaeplastida,0.9970485
cefd772c8ba271fc75492d41ff67d76dbd18e4dc,89,206,468,27999,6,296,334,1720,2347,...,0,2979,3991,207,680,19895,0,5,Eukaryota;Stramenopiles;Ochrophyta;Bacillariophyta;Bacillariophyta_X;Polar-centric-Mediophyceae,0.9999457
ae9f200e11d4c6035213bf838782d6d106acd19c,550,141,45,62,78,3060,59,27,439,...,81,210,5,18,24,3,18302,0,Eukaryota;Alveolata;Dinoflagellata;Dinophyceae;Gonyaulacales;Ceratiaceae;Tripos,0.9998879


In [17]:
# Comile ASV results with reference sequence
asv_wtax <- data.frame(taxa_pr2) %>% 
    rownames_to_column(var = "ReferenceSequence") %>% 
    right_join(fna_df) %>% 
    unite(Taxon_dada2_boot0, Kingdom:Species) %>% 
### MODIFY CONFIDENCE ID FROM DADA2 ####
    left_join(asv_table) %>% 
    select(Feature.ID, Taxon_qiime2 = Taxon, Conf_qiime2 = Confidence, Taxon_dada2_boot0, everything()) %>% 
    data.frame
head(asv_wtax)

Joining, by = "ReferenceSequence"
Joining, by = "Feature.ID"


Feature.ID,Taxon_qiime2,Conf_qiime2,Taxon_dada2_boot0,ReferenceSequence,AP.AV17.AV180216.18S,AP.AV23.AV180316.18S,AP.AV29.AV180420.18S,AP.AV41.AV180518.18S,AV12.18S,...,AV59.18S,AV66.18S,AV69.18S,AV72.18S,AV75.18S,AV78.18S,AV81.18S,AV84.18S,AV90.18S,NEGPCR18SControl.18S
fa846b3096a42b1f708c07aa27fd22f5,Eukaryota;Alveolata;Dinoflagellata;Dinophyceae;Gymnodiniales;Gymnodiniaceae;Margalefidinium;Margalefidinium_fulvescens;,0.8200643,Eukaryota_Alveolata_Dinoflagellata_Dinophyceae_Gymnodiniales_Gymnodiniaceae_Margalefidinium_Margalefidinium_fulvescens,AGCTCCAATAGCGTATATTAAAGTTGTTGCGGTTAAAAAGCTCGTAGTTGGATTTCCGTTCGAGTTCGTACCTCCCCTTGGGGGTTGGTGTCGAGCTCGAGCCTTTCTGGGTGTATACGTGCGTACTTCATTGTATGACGTATTCAACCCGGACTTTTACTTTGAGGAAATTAGAGTGTTTCAGGCAGGCAAACGCCTTGAATACATTAGCATGGAATAATAAGATAAGACTTTGGTCTTGTTTGTTGGTTCATAGACCGAAGTAATGATTAATAGGGATAGTTGGGGGCATTCGTATTTAACTGTCAGAGGTGAAATTCTTGGATTTGTTAAAGACGAACTACTGCGAAAGCATTTGCCAAGGATGTTTTCA,571,337,0,0,1176,...,140,23,76,2925,131694,33199,8219,787,6845,0
219399e2226f5e6d0bf425a5615e4822,Eukaryota;Alveolata;Dinoflagellata;Dinophyceae,0.9999630,Eukaryota_Alveolata_Dinoflagellata_Dinophyceae_Gymnodiniales_Gymnodiniaceae_Gyrodinium_Gyrodinium_spirale,AGCTCCAATAGCGTATATTAAAGTTGTTGCGGTTAAAAAGCTCGTAGTTGGATTTCTGCTGAGGACGACCGGTCCGCCCTCTGGGTGAGTATCTGGCTTGGCCTTGGCATCTTCTTGGAGAACGTATCTGCACTTGACTGTGTGGTGCGGTATCCAGGACTTTTACTTTGAGGAAATTAGAGTGTTTCAAGCAGGCACACGCCTTGAATACATTAGCATGGAATAATAAGATAGGACCTTGGTTCTATTTTGTTGGTTTCTAGAGCTGAGGTAATGATTAATAGGGATAGTTGGGGGCATTCGTATTTAACTGTCAGAGGTGAAATTCTTGGATTTGTTAAAGACGGACTACTGCGAAAGCATTTGCCAAGGATGTTTTCA,0,0,6844,3675,0,...,0,0,0,0,0,0,0,0,0,0
78951b2d4ca30531381672a30be0978a,Eukaryota;Alveolata;Dinoflagellata;Dinophyceae,0.9999875,Eukaryota_Alveolata_Dinoflagellata_Dinophyceae_Gymnodiniales_Gymnodiniaceae_Gyrodinium_Gyrodinium_fusiforme,AGCTCCAATAGCGTATATTAAAGTTGTTGCGGTTAAAAAGCTCGTAGTTGGATTTCTGCTGAGGACGACCGGTCCGCCCTCTGGGTGAGTATCTGGCTTGGCCTTGGCATCTTCTTGGAGAACGTAGCTGCACTTGACTGTGTGGTGCGGTATCCAGGACTTTTACTTTGAGGAAATTAGAGTGTTTCAAGCAGGCACACGCCTTGAATACATTAGCATGGAATAATAAGATAGGACCTTGGTTCTATTTTGTTGGTTTCTAGAGCTGAGGTAATGATTAATAGGGATAGTTGGGGGCATTCGTATTTAACTGTCAGAGGTGAAATTCTTGGATTTGTTAAAGACGGACTACTGCGAAAGCATTTGCCAAGGATGTTTTCA,464,316,2831,757,163,...,567,103,0,739,1443,130,293,594,2256,0
9ddce3e7a0d38ffd2f5bacbcfe906fbf,Eukaryota;Alveolata;Dinoflagellata;Dinophyceae;Gonyaulacales;Ceratiaceae;Tripos,0.9998879,Eukaryota_Alveolata_Dinoflagellata_Dinophyceae_Gonyaulacales_Ceratiaceae_Tripos_Tripos_fusus,AGCTCCAATAGCGTATATTAAAGTTGTTGCGGTTAAAAAGCTCGTAGTTGGATTTCTGCTGAAGCAAACCGGTCCGCCCTCTGGGTGAGCATCTGGCTTTATTTTGGCATATGCTTAGACTTTGCAGCTGCACTTGACTGTGTGGTGTGAAGGCTAAGCCATTTACTTTGAGGAAATCAGAGTGTTTCAAGCAGGCAATTGCCTTGAATACACTAGCATGGAATAATATGATATGACTGTGGTTTTATTTTGTTGGCTTCTAGAATTAGAGTAATGGTTAATAGGGATAGTTGGGGGCATTCATATTTAACTGTCAGAGGTGAAATTCTTGGATTTGTTAAGGATGAACGACTGCGAAAGCATTTGCCAAGGATGCTTTCA,452,0,0,0,63,...,1922,521,23321,5484,818,0,145,47,174,0
5b2f9bda94dd54330d8de22256613968,Eukaryota;Alveolata;Dinoflagellata;Dinophyceae,0.9982531,Eukaryota_Alveolata_Dinoflagellata_Dinophyceae_Gymnodiniales_Gymnodiniaceae_Gyrodinium_Gyrodinium_helveticum,AGCTCCAATAGCGTATATTAAAGTTGTTGCGGTTAAAAAGCTCGTAGTTGGATTTCTGCTGAGGACGACCGGTCCGCCCTCCGGGTGAGCATCTGGTTCGGCCTTGGCATCTTCTTGGTGAACGTATCTGCACTTGACTGTGTGGTGCGGTACCCAGGACTTTTACTTTGAGGAAATTAGAGTGTTTCAAGCAGGCATACGCCTTGAATACATTAGCATGGAATAATAAGATAGGACCTCGGTTCTATTTTGTTGGTTTCTAGAGCTGAGGTAATGATTAATAGGGATAGTTGGGGGCATTCGTATTTAACTGTCAGAGGTGAAATTCTTGGATTTGTTAAAGACGGACTACTGCGAAAGCATTTGCCAAGGATGTTTTCA,542,247,0,299,264,...,120,0,0,0,0,141,86,283,525,0
6b394e36e1bef9a54ee149410df5edd4,Eukaryota;Archaeplastida,0.9964566,Eukaryota_Archaeplastida_Chlorophyta_Mamiellophyceae_Mamiellales_Mamiellaceae_Micromonas_Micromonas_bravo_B1,AGCTCCAATAGCGTATATTTAAGTTGTTGCAGTTAAAAAGCTCGTAGTTGGATTTCGGTTGAGAACGGCCGGTCCGCCGTTTGGTGTGCACTGGCTGGTTTCAACTTCCTGTAGAGGACGCGCTCTGGCTTCATCGCTGGACGCGGAGTCTACGTGGTTACTTTGAAAAAATTAGAGTGTTCAAAGCGGGCTTACGCTTGAATATTTCAGCATGGAATAACACTATAGGACTCCTGTCCTATTTCGTTGGTCTCGGGACGGGAGTAATGATTAAGAGGAACAGT

In [ ]:
# Comile OTU results with reference sequence
otu_wtax <- data.frame(taxa_pr2_otu) %>% 
    rownames_to_column(var = "ReferenceSequence") %>% 
    right_join(fna_df_otu) %>% 
    unite(Taxon_dada2_boot0, Kingdom:Species) %>% 
    select(Conf_dada2_boot0 = Confidence, everything()) %>% 
### MODIFY CONFIDENCE ID FROM DADA2 ####
    left_join(otu_table) %>% 
    select(Feature.ID, Taxon_qiime2 = Taxon, Conf_qiime2 = Confidence, Taxon_dada2_boot0, everything()) %>% 
    data.frame
head(otu_wtax)

In [ ]:
write_delim(asv_wtax, path = "/vortexfs1/omics/huber/shu/slo-pier-weekly/slo-pier-ASV-counttable-wtax-13-08-2020.txt", delim = "\t")
write_delim(otu_wtax, path = "/vortexfs1/omics/huber/shu/slo-pier-weekly/slo-pier-OTU-counttable-wtax-13-08-2020.txt", delim = "\t")